In [1]:
# Q1
import cv2 as cv
import numpy as np

image = cv.imread("/home/pks/Downloads/Assignment/IVP/Assign2/Image1.jpg", 0)
image2 = cv.imread("/home/pks/Downloads/Assignment/IVP/Assign2/Image2.jpg", 0)
# res = cv.matchTemplate(image1,image2, cv.TM_CCOEFF_NORMED)
cv.imshow("window1", image)
cv.waitKey(0)
cv.imshow("window2", image2)
cv.waitKey(0)

r, c = image2.shape
final_img = np.float64(np.copy(image))
image1 = cv.copyMakeBorder(image, 0, r, 0, c, cv.BORDER_REPLICATE)
row, col = image1.shape
mean2 = sum(sum(image2))/(r*c)

normal_image2 = np.float64(np.copy(image2))
normal_image2 = normal_image2 - mean2

for i in range(row-r):
    for j in range(col-c):
        img_array = np.float64(image1[i:i+r, j:j+c])
        mean1 = sum(sum(img_array))/(r*c)
        normal_image1 = img_array - mean1
        
        upper = sum(sum(normal_image1 * normal_image2))
        lower = (sum(sum(normal_image1**2)) * sum(sum(normal_image2**2)))**0.5
        
        final_img[i][j] = upper/lower
        
cv.normalize(final_img, final_img, 0, 255, cv.NORM_MINMAX, dtype=cv.CV_8U)
rf, cf = final_img.shape

min_val, max_val, min_loc, max_loc = cv.minMaxLoc(final_img)
top_left = max_loc
bottom_right = (top_left[0] + c, top_left[1] + r)

cv.rectangle(image, top_left, bottom_right, 255, 2)

cv.imshow("Cor", image)
cv.waitKey(0)

cv.destroyAllWindows()

/home/pks/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
h = cv.calcHist([image2], [0], None, [256], [0,256])

In [ ]:
plt.plot(h, color='g')

In [4]:
# Q4

import cv2 as cv
import numpy as np

image = cv.imread("/home/pks/Downloads/Assignment/IVP/Assign2/e.jpg", 0)

'''
Adding gaussian noise
'''
image_gauss = np.float64(np.copy(image))
mean, variance = tuple(map(float, input("Enter mean and variance (Gaussian Noise) : ").split()))

sigma = variance**0.5             # Root square
noise = np.random.normal(mean, sigma, image.shape)
final_gauss = np.zeros(image.shape, np.float64)
final_gauss = image_gauss + noise

# Scaling down the noisy image to uint8
cv.normalize(final_gauss, final_gauss, 0, 255, cv.NORM_MINMAX, dtype=-1)
final_gauss = final_gauss.astype(np.uint8)


'''
Adding salt and pepper noise
'''
salt_prob = float(input("Enter the probability of salt noise components : "))
pepper_prob = 1.0 - salt_prob
final_sp = np.copy(image)
row, col = image.shape

# Adding salt and pepper
for i in range(row):
    for j in range(col):
        rand_int = np.random.random()
        
        if rand_int < salt_prob:
            final_sp[i][j] = 0
        elif rand_int > pepper_prob:
            final_sp[i][j] = 255

# Displating images
cv.imshow("Original", image)
cv.imshow("Gaussian", final_gauss)
cv.imshow("Salt & Pepper", final_sp)

cv.waitKey(0)
cv.destroyAllWindows()

Enter mean and variance (Gaussian Noise) : 0 0.5
Enter the probability of salt noise components : 0.02


In [5]:
# Q5
import cv2 as cv
import numpy as np

def max_filter(img, kernel_size):
    skip = kernel_size//2
    final_img = np.copy(img)
    img = cv.copyMakeBorder(img, skip, skip, skip, skip, cv.BORDER_REPLICATE)
    row, col = img.shape
    
    for i in range(skip, row-skip):
        for j in range(skip, col-skip):
            img_array = img[i-skip:i+skip+1, j-skip:j+skip+1]
            max_val = np.max(img_array)
            final_img[i-skip][j-skip] = max_val
            
    return final_img

def min_filter(img, kernel_size):
    skip = kernel_size//2
    final_img = np.copy(img)
    img = cv.copyMakeBorder(img, skip, skip, skip, skip, cv.BORDER_REPLICATE)
    row, col = img.shape
    
    for i in range(skip, row-skip):
        for j in range(skip, col-skip):
            img_array = img[i-skip:i+skip+1, j-skip:j+skip+1].ravel()
            min_val = np.min(img_array)
            final_img[i-skip][j-skip] = min_val
            
    return final_img

def median_filter(img, kernel_size):
    skip = kernel_size//2
    final_img = np.copy(img)
    img = cv.copyMakeBorder(img, skip, skip, skip, skip, cv.BORDER_REPLICATE)
    row, col = img.shape
    
    for i in range(skip, row-skip):
        for j in range(skip, col-skip):
            img_array = img[i-skip:i+skip+1, j-skip:j+skip+1].ravel()
            median_val = np.median(img_array)
            final_img[i-skip][j-skip] = median_val
            
    return final_img

def apply_filter(img, kernel, norm_coff=1):
    pad = 1
    final_img = np.float64(np.copy(img))
    img = cv.copyMakeBorder(img, pad, pad, pad, pad, cv.BORDER_REPLICATE)
    
    row, col = img.shape
    for i in range(pad, row-pad):
        for j in range(pad, col-pad):
            img_array = np.float64(img[i-pad:i+pad+1, j-pad:j+pad+1])
            # Multiplying with filter
            img_array = img_array * kernel
            pixel_sum = (sum(sum(img_array)))/norm_coff
            # Storing new pixel value image
            final_img[i-pad][j-pad] = pixel_sum
            
    cv.normalize(final_img, final_img, 0, 255, cv.NORM_MINMAX, dtype=cv.CV_8U)
    
    return final_img.astype(np.uint8)

if __name__ == '__main__':
    path = "/home/pks/Downloads/Assignment/IVP/Assign2/"
    
    gaussian_kernel = np.array([[1,2,1], [2,4,2], [1,2,1]])
    mean_kernel = np.array([[1,1,1], [1,1,1], [1,1,1]])
    
    a = cv.imread(path+"a.jpg", 0)
    b = cv.imread(path+"b.jpg", 0)
    c = cv.imread(path+"c.jpg", 0)
    d = cv.imread(path+"d.jpg", 0)
    e = cv.imread(path+"e.jpg", 0)
    f = cv.imread(path+"f.jpg", 0)
    g = cv.imread(path+"g.jpg", 0)
    
    # b.jpg ()
    cv.imshow("Original-b", b)
    cv.waitKey(0)
    b_img = apply_filter(b, gaussian_kernel, 16)
    gamma, C = 1.2, 255
    b_img = np.array(C*((b_img/255)**gamma), dtype=np.uint8)
    cv.imshow("b.jpg (gaussian)", b_img)
    cv.waitKey(0)

    # c.jpg
    cv.imshow("Original-c", c)
    cv.waitKey(0)
    c_img = median_filter(c, 5)
    cv.imshow("c.jpg (median)", c_img)
    cv.waitKey(0)

    # d.jpg
    cv.imshow("Original-d", d)
    cv.waitKey(0)
    d_img = median_filter(d, 5)
    cv.imshow("d.jpg (median)", d_img)
    cv.waitKey(0)
    

    # f.jpg (Pepper noise)
    cv.imshow("Original-f", f)
    cv.waitKey(0)
    f_img = max_filter(f, 3)
    cv.imshow("f.jpg (max)",f_img)
    cv.waitKey(0)

    # g.jpg (Salt noise)
    cv.imshow("Original-g", g)
    cv.waitKey(0)
    g_img = min_filter(g, 3)
#     cv.imshow("g.jpg (min)", g_img)
#     cv.waitKey(0)
    g_img = max_filter(g_img, 3)
    cv.imshow("g.jpg (min - max)", g_img)
    cv.waitKey(0)
    
    cv.destroyAllWindows()

In [2]:
# Q2
import cv2 as cv
import numpy as np

path = "/home/pks/Downloads/Assignment/IVP/Assign2/"

def apply_filter(img, kernel, norm_coff=1):
    pad = 1
    final_img = np.float64(np.copy(img))
    img = cv.copyMakeBorder(img, pad, pad, pad, pad, cv.BORDER_REPLICATE)
    
    row, col = img.shape
    for i in range(pad, row-pad):
        for j in range(pad, col-pad):
            img_array = np.float64(img[i-pad:i+pad+1, j-pad:j+pad+1])
            # Multiplying with filter
            img_array = img_array * kernel
            pixel_sum = (sum(sum(img_array)))/norm_coff
            # Storing new pixel value image
            final_img[i-pad][j-pad] = int(pixel_sum)
            
    return final_img

def apply_rob_filter(img, kernel, norm_coff=1):
    pad = 1
    final_img = np.float64(np.copy(img))
    img = cv.copyMakeBorder(img, 0, pad, 0, pad, cv.BORDER_REPLICATE)
    
    row, col = img.shape
    for i in range(row-pad):
        for j in range(col-pad):
            img_array = np.float64(img[i:i+pad+1, j:j+pad+1])
            # Multiplying with filter
            img_array = img_array * kernel
            pixel_sum = (sum(sum(img_array)))/norm_coff
            # Storing new pixel value image
            final_img[i][j] = int(pixel_sum)
            
    return final_img

if __name__ == '__main__':
    prewitt_kernel_x = np.array([[-1,0,1], [-1,0,1], [-1,0,1]])
    prewitt_kernel_y = np.array([[1,1,1], [0,0,0], [-1,-1,-1]])

    sobel_kernel_x = np.array([[-1,0,1], [-2,0,2], [-1,0,1]])
    sobel_kernel_y = np.array([[1,2,1], [0,0,0], [-1,-2,-1]])

    laplacian_kernel = np.array([[0,-1,0], [-1,4,-1], [0,-1,0]])
    laplacian_kernel_diag = np.array([[-1,-1,-1], [-1,8,-1], [-1,-1,-1]])
    
    roberts_kernel_x = np.array([[0,1], [-1,0]])
    roberts_kernel_y = np.array([[1,0], [0,-1]])
    
    image = cv.imread(path + "barbara.jpg", 0)
    cv.imshow("Original", image)
    cv.waitKey(0)
    
    # Applying Prewitt operator
    prewitt_image_x = apply_filter(image, prewitt_kernel_x, 6)
    prewitt_image_y = apply_filter(image, prewitt_kernel_y, 6)
    
    prewitt_image = cv.add(abs(prewitt_image_x), abs(prewitt_image_y))
    cv.normalize(prewitt_image, prewitt_image, 0, 255, cv.NORM_MINMAX, dtype=-1)
    prewitt_image = prewitt_image.astype(np.uint8)
    
    # Applying Sobel operator
    sobel_image_x = apply_filter(image, sobel_kernel_x, 8)
    sobel_image_y = apply_filter(image, sobel_kernel_y, 8)
    
    sobel_image = cv.add(abs(sobel_image_x), abs(sobel_image_y))
    cv.normalize(sobel_image, sobel_image, 0, 255, cv.NORM_MINMAX, dtype=-1)
    sobel_image = sobel_image.astype(np.uint8)
    
    # Applyig Laplacian Filter
    laplacian_image = apply_filter(image, laplacian_kernel)
    cv.normalize(laplacian_image, laplacian_image, 0, 255, cv.NORM_MINMAX, dtype=-1)
    laplacian_image = laplacian_image.astype(np.uint8)
    
    # laplacian_image = cv.filter2D(image, -1, laplacian_kernel_diag, cv.BORDER_DEFAULT)
    laplacian_image_diag = apply_filter(image, laplacian_kernel_diag)
    cv.normalize(laplacian_image_diag, laplacian_image_diag, 0, 255, cv.NORM_MINMAX, dtype=-1)
    laplacian_image_diag = laplacian_image_diag.astype(np.uint8)
    
    # Applying Roberts operator
    roberts_image_x = apply_rob_filter(image, roberts_kernel_x, 1)
    roberts_image_y = apply_rob_filter(image, roberts_kernel_y, 1)
    
    roberts_image = cv.add(abs(roberts_image_x), abs(roberts_image_y))
    cv.normalize(roberts_image, roberts_image, 0, 255, cv.NORM_MINMAX, dtype=-1)
    roberts_image = roberts_image.astype(np.uint8)
    
    # Displaying Images after applying filter
    cv.imshow("prewitt", prewitt_image)
    cv.waitKey(0)
    
    cv.imshow("Sobel", prewitt_image)
    cv.waitKey(0)
    
    cv.imshow("Laplacian", laplacian_image)
    cv.waitKey(0)
    
    cv.imshow("Laplacian Diag", laplacian_image_diag)
    cv.waitKey(0)
    
    cv.imshow("Roberts", roberts_image)
    cv.waitKey(0)
    
    cv.destroyAllWindows()

In [3]:
# Q3
import cv2 as cv
import numpy as np

path = "/home/pks/Downloads/Assignment/IVP/Assign2/"

if __name__ == '__main__':
    image = cv.imread(path + "skeleton_orig.tif", 0)
    
    # Original Image (a)
    cv.imshow("Original", image)
    cv.waitKey(0)
    
    # Laplacian Filter (c)
    laplacian_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    laplacian_image = cv.filter2D(image, -1, laplacian_kernel, cv.BORDER_DEFAULT)
    
    cv.imshow("Laplacian on org", laplacian_image)
    cv.waitKey(0)
    
    # Sobel Filter (d)
    sobel_x = cv.Sobel(image, cv.CV_16S, 1, 0, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    sobel_y = cv.Sobel(image, cv.CV_16S, 0, 1, ksize=3, scale=1, delta=0, borderType=cv.BORDER_DEFAULT)
    
    abs_sobel_x = cv.convertScaleAbs(sobel_x)
    abs_sobel_y = cv.convertScaleAbs(sobel_y)    
    sobel_image = cv.addWeighted(abs_sobel_x, 0.5, abs_sobel_y, 0.5, 0)
    
    cv.imshow("Sobel on org", sobel_image)
    cv.waitKey(0)
    
    # Average Filter (e)
    smooth_image = cv.blur(sobel_image, (5,5))
    
    cv.imshow("Smoothed sobel", smooth_image)
    cv.waitKey(0)
    
    # Mask (f)
    mask_image = np.uint16(np.copy(laplacian_image))
    mask_image = np.multiply(mask_image, smooth_image)
    cv.normalize(mask_image, mask_image, 0, 255, cv.NORM_MINMAX, dtype=-1)
    mask_image = mask_image.astype(np.uint8)
    
    cv.imshow("Mask (laplacian x smooth)", mask_image)
    cv.waitKey(0)
    
    # Sharpened Image (g)
    sharp_image = cv.add(image, mask_image)
    
    cv.imshow("Sharp (org + mask)", sharp_image)
    cv.waitKey(0)
    
    # Power Law transformation
    gamma, c = 0.7, 255
    final_image = np.array(c*((sharp_image/255)**gamma), dtype=np.uint8)
    
    # Displaying Enhanced Image
    cv.imshow("Final Image (power law on sharp)", final_image)
    cv.waitKey(0)
    
    cv.destroyAllWindows()

In [6]:
# Q6
import cv2 as cv
import numpy as np

path = "/home/pks/Downloads/Assignment/IVP/Assign2/"

def mean_sq_err(img1, img2):
    row, col = img1.shape
    
    diff = np.float64(np.copy(img1))
    diff = diff - img2
    diff = diff**2
    
    error = (sum(sum(diff)))/(row*col)
    
    return error

def error_image(img1, img2):
    diff = np.float64(np.copy(img1))
    diff = diff - img2
    
    cv.normalize(diff, diff, 0, 255, cv.NORM_MINMAX, dtype=-1)
    
    return diff.astype(np.uint8)

if __name__ == '__main__':
    noisy_image = cv.imread(path + "img1noisy.tif", 0)
    org_image = cv.imread(path + "img1original.tif", 0)
    
    cv.imshow("Org", org_image)
    cv.waitKey(0)
    
    cv.imshow("Noisy", noisy_image)
    cv.waitKey(0)
    
    row, col = noisy_image.shape
    
    print("Mean-square errors : ")
    
    print("Before filtering - ", mean_sq_err(org_image, noisy_image))
    
    imp_l = cv.medianBlur(noisy_image[:, :col//2], 3)
    imp_img_l = np.copy(noisy_image)
    imp_img_l[:, :col//2] = imp_l
    
    cv.imshow("Median-l", imp_img_l)
    cv.waitKey(0)
    
    print("Filtering left-part with median filter - ", mean_sq_err(org_image, imp_img_l))
    
    imp_r = cv.medianBlur(noisy_image[:, col//2:], 3)
    imp_img_r = np.copy(noisy_image)
    imp_img_r[:, col//2:] = imp_r
    
    cv.imshow("Median-r", imp_img_r)
    cv.waitKey(0)
    
    print("Filtering right-part with median filter - ", mean_sq_err(org_image, imp_img_r))
    
    gaus_l = cv.blur(noisy_image[:, :col//2], (3,3))
    gaus_img_l = np.copy(noisy_image)
    gaus_img_l[:, :col//2] = gaus_l
    
    cv.imshow("Mean-l", gaus_img_l)
    cv.waitKey(0)
    
    print("Filtering left-part with mean filter - ", mean_sq_err(org_image, gaus_img_l))
    
    gaus_r = cv.blur(noisy_image[:, col//2:], (3,3))
    gaus_img_r = np.copy(noisy_image)
    gaus_img_r[:, col//2:] = gaus_r
    
    cv.imshow("Mean-r", gaus_img_r)
    cv.waitKey(0)
    
    print("Filtering right-part with mean filter - ", mean_sq_err(org_image, gaus_img_r))
    
    imp_img = cv.medianBlur(noisy_image, 3)
    gaus_img = cv.blur(noisy_image, (3,3))
    
    error_imp = mean_sq_err(org_image, imp_img)
    error_gaus = mean_sq_err(org_image, gaus_img)
    
    if error_imp < error_gaus:
        
        print("\nMedian filter is the best for this scenario")
        
        cv.imshow("Median_Filter", imp_img)
        cv.waitKey(0)
        
        cv.imshow("Error", error_image(org_image, imp_img))
        cv.waitKey(0)
        
    else:
        
        print("\nMean filter is the best for this scenario")
        
        cv.imshow("Mean_Filter", gaus_img)
        cv.waitKey(0)
        
        cv.imshow("Error", error_image(org_image, gaus_img))
        cv.waitKey(0)
    
    cv.destroyAllWindows()

Mean-square errors : 
Before filtering -  490.78224182128906
Filtering left-part with median filter -  276.11766052246094
Filtering right-part with median filter -  279.47142028808594
Filtering left-part with mean filter -  305.1565246582031
Filtering right-part with mean filter -  266.0408172607422

Median filter is the best for this scenario
